In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(review, "html.parser")

In [6]:
import re

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [11]:
ps = PorterStemmer()

In [14]:
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size = 0.3, random_state = 0)

In [24]:
y_train = (y_train.replace({'positive': 1, 'negative': 0})).values
y_test  = (y_test.replace({'positive': 1, 'negative': 0})).values

In [25]:
corpus_train = []
corpus_test  = []

for i in range(x_train.shape[0]):
    soup = BeautifulSoup(x_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)

In [26]:
for j in range(x_test.shape[0]):
    soup = BeautifulSoup(x_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)

In [29]:
from sklearn.svm import LinearSVC
linear_svc = LinearSVC(C=0.5, random_state=42)

In [33]:
linear_svc.fit(tfidf_vec_train, y_train)

LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [34]:
predict = linear_svc.predict(tfidf_vec_test)

In [35]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [37]:
print("Classification Report: \n", classification_report(y_test, predict,target_names=['Negative','Positive']))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.91      0.88      0.90      7540
    Positive       0.88      0.91      0.90      7460

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



In [38]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predict))

Confusion Matrix: 
 [[6633  907]
 [ 645 6815]]


In [39]:
print("Accuracy: \n", accuracy_score(y_test, predict))

Accuracy: 
 0.8965333333333333
